# Capstone project - The battle of Neighbourhoods

This notebook is mainly for the Applied Data Science Capstone project

In [1]:
import numpy as np
import pandas as pd

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Introduction/Business Problem

Toronto, the capital city of Ontario, is the most popular city in Canada. It is a multicultural city, and the financial and commercial centre of the country. Each year tons of  thousands of newcomers all over the world choose Toronto as their new home. 

With the increase demand in housing market, the price keeps going up. Residents wish to live in an area where the housing price is relatively low and plenty of social venues are available around the neighbourhoods. In this project I will segment and cluster neighbourhoods in Toronto according to the housing prices and venues density. Residents can have a more clear idea with pricing and living environment of each neighbourhood. 

## Data Section

-Foursquare location data will be used to get the venues of borough in Toronto. 

-The postal code and borough information can be scraped from the Wikipedia website. 

-The housing price information can be collected from Toronto CTVnews website

-Geocoder Python Package can be used to get coordinates of each borough.